In [1]:
import datetime
import numpy as np
import pandas as pd
from pandas import Series
from itertools import groupby
import matplotlib.pyplot as plt

%matplotlib inline  

In [2]:
df = pd.read_csv('../../data/train.csv')
df['original'] = True
ids = df['KPI ID'].unique()

In [3]:
df.head()

,timestamp,value,label,KPI ID,original
0,1493568000,1.901639,0,02e99bd4f6cfb33f,True
1,1493568060,1.786885,0,02e99bd4f6cfb33f,True
2,1493568120,2.000000,0,02e99bd4f6cfb33f,True
3,1493568180,1.885246,0,02e99bd4f6cfb33f,True
4,1493568240,1.819672,0,02e99bd4f6cfb33f,True


In [4]:
df['timestamp'].describe()

count    2.476315e+06
mean     1.495904e+09
std      4.479875e+06
min      1.467302e+09
25%      1.494256e+09
50%      1.496356e+09
75%      1.498562e+09
max      1.502447e+09
Name: timestamp, dtype: float64

In [5]:
# train set
dfs = []
for _id in ids:
    df_tmp = df[df['KPI ID'] == _id]
    df_tmp['datetime'] = pd.to_datetime(df_tmp['timestamp'], unit = 's')
    df_tmp = df_tmp.set_index('datetime')
    df_tmp = df_tmp.sort_index()
    
    df_tmp_value = df_tmp['value'].resample('1T').interpolate()
    df_tmp_label = df_tmp['label'].resample('1T').ffill()

    df_tmp = pd.concat([df_tmp_value, df_tmp_label], axis=1, join='inner')
    df_tmp.insert(1, 'KPI ID', _id)
    df_tmp['timestamp'] = (df_tmp_value.index.astype('int64') // 1e9).astype('int64')
    
    df_tmp['id'] = df_tmp['KPI ID'] + '-' + df_tmp.index.astype('str')
    df_tmp = df_tmp.set_index('id')
    dfs.append(df_tmp)

df_upsampled = pd.concat(dfs)

/Users/alex/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/alex/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# test set
dfs = []
for _id in ids:
    df_tmp = df[df['KPI ID'] == _id]
    df_tmp['datetime'] = pd.to_datetime(df_tmp['timestamp'], unit = 's')
    df_tmp = df_tmp.set_index('datetime')
    df_tmp = df_tmp.sort_index()
    
    df_tmp_value = df_tmp['value'].resample('1T').interpolate()
    df_tmp_label = df_tmp['label'].resample('1T').ffill()

    df_tmp = pd.concat([df_tmp_value, df_tmp_label], axis=1, join='inner')
    df_tmp.insert(1, 'KPI ID', _id)
    df_tmp['timestamp'] = (df_tmp_value.index.astype('int64') // 1e9).astype('int64')
    
    df_tmp['id'] = df_tmp['KPI ID'] + '-' + df_tmp.index.astype('str')
    df_tmp = df_tmp.set_index('id')
    dfs.append(df_tmp)

df_upsampled = pd.concat(dfs)

In [6]:
# normalization
for _id in ids:
    df_tmp = df_upsampled[df_upsampled['KPI ID'] == _id]
    df_tmp['value_diff'] = df_tmp['value'].diff()
    
    df_upsampled.loc[df_upsampled['KPI ID'] == _id, 'value'] = (df_tmp['value'] - df_tmp['value'].mean()) / df_tmp['value'].std()
    df_upsampled.loc[df_upsampled['KPI ID'] == _id, 'value_diff'] = (df_tmp['value_diff'] - df_tmp['value_diff'].mean()) / df_tmp['value_diff'].std()

/Users/alex/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
df_upsampled.loc[df_upsampled['value_diff'].isnull(), 'value_diff'] = 0.0

In [8]:
df_upsampled.head()

,value,KPI ID,label,timestamp,value_diff
id,,,,,
02e99bd4f6cfb33f-2017-04-30 16:00:00,-0.080901,02e99bd4f6cfb33f,0,1493568000,0.000000
02e99bd4f6cfb33f-2017-04-30 16:01:00,-0.256014,02e99bd4f6cfb33f,0,1493568060,-0.570665
02e99bd4f6cfb33f-2017-04-30 16:02:00,0.069196,02e99bd4f6cfb33f,0,1493568120,1.059743
02e99bd4f6cfb33f-2017-04-30 16:03:00,-0.105917,02e99bd4f6cfb33f,0,1493568180,-0.570665
02e99bd4f6cfb33f-2017-04-30 16:04:00,-0.205981,02e99bd4f6cfb33f,0,1493568240,-0.326104
